# Motor Vehicle Accident XML Check

This notebook checks the integrity of the periodic Motor Vehicle Accident (MVA) report XML files we receive into `\\tssrv7\CollisionsProcessed\AtSceneXmlBackups`.

In [1]:
%matplotlib inline

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import configparser
import pathlib
import psycopg2
import psycopg2.extras as ppg2extras
import re

# Default mpl colours.
prop_cycle = plt.rcParams['axes.prop_cycle']
mpl_def_colors = prop_cycle.by_key()['color']

In [2]:
config = configparser.ConfigParser()
config.read(pathlib.Path.home().joinpath('.charlesconfig').as_posix());
postgres_settings = config['POSTGRES']

## Mount Drive

Following [this answer](https://superuser.com/a/1261563), mounted drive in WSL using:

```
sudo mkdir /mnt/ldrive
sudo mount -t drvfs L: /mnt/ldrive
```

Tested

```
sudo umount -l /mnt/ldrive
```

for unmounting.

## Check File Structure

In [3]:
rootfolder = '/mnt/ldrive/AtSceneXmlBackups/'
xml_folders = os.listdir(rootfolder)
filenames = {fn: os.listdir(rootfolder + fn) for fn in xml_folders}

To load each XML we use the element tree:

In [4]:
import xml.etree.ElementTree as et

def xmldfs_step(node, row_dict):
    if node.text is not None:
        if len(node.text.split()):
            row_dict[node.tag] = node.text
    for child in node.getchildren():
        xmldfs_step(child, row_dict)

def extract_xml_row(filepath):
    xtree = et.parse(filepath)
    row_dict = {}
    xmldfs_step(xtree.getroot(), row_dict)
    return row_dict

In [5]:
%%time
dicts = [extract_xml_row(rootfolder + '401XmlLoadDir-20160309/' + x)
         for x in filenames['401XmlLoadDir-20160309'][:800]]

CPU times: user 1.38 s, sys: 3.59 s, total: 4.97 s
Wall time: 1min 14s


Since there are ~8000 files in that folder it'll take us ~15 min to go through them all, plus the stuff that's been extracted isn't necessarily useful unless we suspect the ACC number has been tampered with.

In [6]:
def extract_accloc(xtree):
    accloc_node = (xtree.getroot().getchildren()[0]
                   .find('ACCIDENT').getchildren()[0]
                   .getchildren()[0].find('AccidentLocation-Box-1'))

    if accloc_node is None:
        raise ValueError('could not find AccidentLocation-Box-1!')

    try:
        return int(accloc_node.text)
    except Exception:
        return -1


def extract_occ_yr(xtree):
    occyr_node = (xtree.getroot().getchildren()[0]
                   .find('OCC_YR'))

    if occyr_node is None:
        raise ValueError('could not find OCC_YR!')

    try:
        return int(occyr_node.text)
    except Exception:
        return -1


def extract_xml_row_short(filepath):
    xtree = et.parse(filepath)
    return {'filepath': filepath,
            'accloc': extract_accloc(xtree),
            'occ_yr': extract_occ_yr(xtree)}

In [7]:
%%time
dicts = [extract_xml_row_short(rootfolder + '401XmlLoadDir-20160309/' + x)
         for x in filenames['401XmlLoadDir-20160309'][:800]]

CPU times: user 1.34 s, sys: 3.05 s, total: 4.39 s
Wall time: 1min 13s


Ahh okay, so the rate-limiting step is either reading in the files in the first place, or generating the parse tree. That's irritating, since the reasonable solution is to only read in those files whose ACCNBs are missing from ACC.

In [8]:
def get_name_info(filename, filepath):
    accyear_str, go_number_str = re.split("_|-", filename[:-4])[-2:]
    try:
        go_number = int(go_number_str)
    except Exception:
        raise ValueError("{0} was converted to {1}, which can't be "
                         "converted to an integer."
                         .format(mystr, go_number))

    try:
        accyear = int(accyear_str[2:])
        assert (accyear // 1000 > 1), ""
    except Exception as exc:
        raise ValueError("{0} was converted to {1}, which can't be "
                         "converted to an integer. Error raised:"
                         .format(mystr, accyear, str(exc)))

    acc_number = 1000000000 * int(accyear_str[-1]) + go_number
    
    return {'fullpath': filepath + "/" + filename,
            'filename': filename,
            'accyear': accyear,
            'go_number': go_number,
            'acc_number': acc_number}

coln_xml_r = [get_name_info(x, fn_key)
              for fn_key in filenames.keys() for x in filenames[fn_key]
              if x[-4:] == '.xml']
coln_xml_r = pd.DataFrame(coln_xml_r)

In [9]:
coln_xml_r_unique = coln_xml_r.groupby(
    ['filename', 'accyear', 'go_number', 'acc_number'])['fullpath'].count()
dups = coln_xml_r_unique[coln_xml_r_unique > 1]
dups = dups.reset_index()['acc_number'].unique()

In [10]:
coln_xml_r.loc[coln_xml_r['acc_number'].isin(dups), :].sort_values('filename')

,fullpath,filename,accyear,go_number,acc_number
0,401XmlLoadDir-20151214/MVA_GO_TP2013-1000069.xml,MVA_GO_TP2013-1000069.xml,2013,1000069,3001000069
309280,401XmlLoadDir-20190403/MVA_GO_TP2013-1000069.xml,MVA_GO_TP2013-1000069.xml,2013,1000069,3001000069
242874,401XmlLoadDir-20190328/MVA_GO_TP2013-1000069.xml,MVA_GO_TP2013-1000069.xml,2013,1000069,3001000069
165145,401XmlLoadDir-20180511/MVA_GO_TP2013-1000069.xml,MVA_GO_TP2013-1000069.xml,2013,1000069,3001000069
101111,401XmlLoadDir-20180416/MVA_GO_TP2013-1000069.xml,MVA_GO_TP2013-1000069.xml,2013,1000069,3001000069
...,...,...,...,...,...
437862,401XmlLoadDir-20210204/MVA_GO_TP2021-9746.xml,MVA_GO_TP2021-9746.xml,2021,9746,1000009746
442571,401XmlLoadDir-20210207/MVA_GO_TP2021-98784.xml,MVA_GO_TP2021-98784.xml,2021,98784,1000098784
334632,401XmlLoadDir-20210126/MVA_GO_TP2021-98784.xml,MVA_GO_TP2021-98784.xml,2021,98784,1000098784
437863,401XmlLoadDir-20210204/MVA_GO_TP2021-99618.xml,MVA_GO_TP2021-99618.xml,2021,99618,1000099618


So there's plenty of cases where MVA reports are sent to us on multiple occasions.

In [11]:
!diff -q /mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20151214/MVA_GO_TP2013-1000069.xml /mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20190328/MVA_GO_TP2013-1000069.xml

Files /mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20151214/MVA_GO_TP2013-1000069.xml and /mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20190328/MVA_GO_TP2013-1000069.xml differ


In [12]:
# olddict = extract_xml_row('/mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20151214/MVA_GO_TP2013-1000069.xml')
# newdict = extract_xml_row('/mnt/ldrive/AtSceneXmlBackups/401XmlLoadDir-20190328/MVA_GO_TP2013-1000069.xml')

# badkeys = []

# for key in olddict.keys():
#     if olddict[key] != newdict[key]:
#         badkeys.append(key)

A closer investigation into these two files (which won't be recorded here since these files contain sensitive information) shows that the differences are due to XML syntax changes, like the inclusion of `xmlns=""`. This makes it generally difficult check the half-million files that may have duplicates.

If we assume these files are all identical, then we can drop duplicates.

In [13]:
coln_xml = coln_xml_r[['accyear', 'go_number', 'acc_number']].drop_duplicates()
coln_xml.reset_index(drop=True, inplace=True)

**Restrict analysis to 2014-2019 only** (since we're more certain of completeness in those 6 years).

In [14]:
coln_xml_20152019 = coln_xml.loc[(coln_xml['accyear'] >= 2014) & (coln_xml['accyear'] < 2020), :].copy()

Obtain corresponding collisions from `collisions.events`. Here we differentiate CRC, TPS and other collision sources using the ACCNB numbering scheme [as described by Jim Millington](https://github.com/CityofToronto/bdit_data-sources/pull/349#issuecomment-803133700).

In [15]:
sql_query = """SELECT collision_no,
       accyear,
       accnb,
       CASE
         WHEN accnb >= 1000000000 THEN 'TPS'
         WHEN accnb >= 100000000 THEN 'CRC'
         ELSE 'DUNNO'
       END data_source
FROM collisions.events
WHERE accdate BETWEEN '2014-01-01' AND '2019-12-31'"""

with psycopg2.connect(**postgres_settings) as db_con:
    df_acc = pd.read_sql(sql_query, db_con)
    df_acc['accyear'] = df_acc['accyear'].astype(int)

In [16]:
df_acc['data_source'].value_counts()

CRC      270936
TPS       71812
DUNNO        84
Name: data_source, dtype: int64

In [17]:
df_merge = pd.merge(coln_xml_20152019, df_acc, how='outer',
                    left_on='acc_number', right_on='accnb',
                    suffixes=('_xml', '_acc'))

Most collisions in ACC but not in the XML folders are from the CRC, which is to be expected. 191 have 10-digit ACCNBs, though, suggesting they're from the TPS:

In [18]:
df_merge.loc[df_merge['accyear_xml'].isna(), 'data_source'].value_counts(dropna=False)

CRC      270936
TPS         245
DUNNO        84
Name: data_source, dtype: int64

Are there any obvious patterns to the ACCNBs and years of these TPS collisions not in the XML files?

In [19]:
df_merge.loc[df_merge['accyear_xml'].isna() & (df_merge['data_source'] == 'TPS'), 'accnb'].astype(int)

115173    4000009168
115174    4000031625
115175    4000047040
115176    4000063626
115177    4000279522
             ...    
352078    9002274826
352079    9002294353
352080    9002389784
352081    9002403050
352082    9002465950
Name: accnb, Length: 245, dtype: int64

In [20]:
(df_merge.loc[df_merge['accyear_xml'].isna() & (df_merge['data_source'] == 'TPS'), :]
 .groupby('accyear_acc')['accyear_acc'].count())

accyear_acc
2014.0    12
2015.0    11
2016.0    49
2017.0    54
2018.0    62
2019.0    57
Name: accyear_acc, dtype: int64

No obvious patterns here - the collisions are distributed over all 5 years. How about the `DUNNO` collisions (ones whose ACCNB number don't suggest either a CRC or TPS source)?

In [21]:
df_merge.loc[(df_merge['data_source'] == 'DUNNO'), 'accnb']

151269          15.0
151270       39858.0
151271       43490.0
151272       45868.0
151273      254616.0
             ...    
297370    19195526.0
297371    19802929.0
297372    19804751.0
297373    19854768.0
297374    98003909.0
Name: accnb, Length: 84, dtype: float64

In [22]:
(df_merge.loc[(df_merge['data_source'] == 'DUNNO'), :]
 .groupby('accyear_acc')['accyear_acc'].count())

accyear_acc
2016.0    24
2017.0    16
2018.0    12
2019.0    32
Name: accyear_acc, dtype: int64

That's interesting - this looks like a data ingestion problem, since the offending collisions come from only three years. Are any of these collisions in the XML files?

In [23]:
sql_query = """SELECT collision_no,
       accyear,
       accnb + 1000000000::bigint * (accyear::integer % 2010) fake_accnb
FROM collisions.events
WHERE accdate BETWEEN '2014-01-01' AND '2019-12-31'
    AND accnb < 100000000"""

with psycopg2.connect(**postgres_settings) as db_con:
    df_dunno = pd.read_sql(sql_query, db_con)
    df_dunno['accyear'] = df_dunno['accyear'].astype(int)

In [24]:
pd.merge(coln_xml_20152019, df_dunno, how='right',
         left_on='acc_number', right_on='fake_accnb',
         suffixes=('_xml', '_acc')).dropna()

,accyear_xml,go_number,acc_number,collision_no,accyear_acc,fake_accnb


So none of the ACCNBs that defy the two naming schemes have obvious corresponding numbers in the XML folders.

Meanwhile, all collisions in the XML folders are from TPS, or aren't included in ACC:

In [25]:
df_merge.loc[~df_merge['accyear_xml'].isna(), 'data_source'].value_counts(dropna=False)

TPS    71567
NaN     9251
Name: data_source, dtype: int64

In [26]:
missing_acc = df_merge.loc[(df_merge['data_source'].isna()), 'acc_number'].astype(int).values

In [27]:
coln_missing_acc = (
    coln_xml_r.loc[coln_xml_r['acc_number'].isin(missing_acc), :]
    .groupby(['filename', 'accyear', 'go_number', 'acc_number'])['fullpath']
    .min()).reset_index()

In [28]:
%%time
row_dicts = []
for i, row in coln_missing_acc.iterrows():
    row_dicts.append(extract_xml_row_short(rootfolder + row['fullpath']))

cma_xmlinfo = pd.DataFrame(row_dicts, index=coln_missing_acc.index)

CPU times: user 16.9 s, sys: 42.5 s, total: 59.4 s
Wall time: 15min 53s


In [29]:
cma_xmlinfo['accloc'].value_counts()

 10    6608
 99     992
 4      835
 1      413
-1      109
 2      108
 3      105
 98      44
 8       26
 6        7
 7        2
 5        1
 9        1
Name: accloc, dtype: int64

So most of these are are class 10 (parking lot), 99 (private driveway) or 4 (at/near private drive). All these can be construed as private property, so we would expect to see them in ACC_ARCHIVE (as mentioned in [Evan's spelunking findings](https://www.notion.so/bditto/CRASH-Data-Intake-Scripts-Spelunking-e2bd8895ce3d40eb9e58f24e0c0c2d76#bd6b7a2f1c664248902c41a8cfd5b3c4)). What concerns me is that, for the same timespan, thare are hundreds of class 99 and 4 collisions in ACC itself. There's also approximately 700 collisions that *should* be in ACC but aren't for some reason, since their codes are 1, 2, etc.

In [30]:
sql_query = """WITH step_1 AS (
	SELECT location_type,
	       COUNT(*) n_loctype
	FROM collisions.events
	WHERE accdate BETWEEN '2014-01-01' AND '2019-12-31'
	GROUP BY location_type
)
SELECT a.location_type,
       b.accloc,
       a.n_loctype
FROM step_1 a
LEFT JOIN collision_factors.accloc b ON a.location_type = UPPER(b.description);"""

with psycopg2.connect(**postgres_settings) as db_con:
    df_accloc = pd.read_sql(sql_query, db_con)
df_accloc

,location_type,accloc,n_loctype
0,NON INTERSECTION,01,238462
1,INTERSECTION RELATED,02,39749
2,AT INTERSECTION,03,48612
3,AT/NEAR PRIVATE DRIVE,04,14791
4,AT RAILWAY CROSSING,05,113
5,UNDERPASS OR TUNNEL,06,88
6,OVERPASS OR BRIDGE,07,140
7,TRAIL,08,36
8,FROZEN LAKE OR RIVER,09,101
9,OTHER,97,95


In [31]:
coln_missing_acc = pd.concat([coln_missing_acc, cma_xmlinfo], axis=1)

In [32]:
coln_missing_acc[['acc_number', 'accloc', 'occ_yr']]

,acc_number,accloc,occ_yr
0,4001008236,3,2014
1,4001071106,4,2014
2,4001099185,1,2014
3,4001164130,-1,2014
4,4011790446,10,2014
...,...,...,...
9246,9000988877,4,2019
9247,9000994345,10,2019
9248,9000995030,10,2019
9249,9000996365,4,2019


In [33]:
df_fulldata = pd.merge(df_merge, coln_missing_acc[['acc_number', 'accloc', 'occ_yr']],
                       how='left', left_on='acc_number', right_on='acc_number')

In [34]:
assert np.all(df_fulldata.loc[~df_fulldata['occ_yr'].isna(), 'accyear_xml']
               == df_fulldata.loc[~df_fulldata['occ_yr'].isna(), 'occ_yr']), (
    "XML filenames and occurrence year entries do not match!")

In [35]:
# df_fulldata.to_hdf('./20210422_mva_check_temp.hdf5', 'table')
# df = pd.read_hdf('./20210422_mva_check_temp.hdf5', 'table')
# assert df.equals(df_fulldata)
# df_fulldata = df

Generate some summary statistics:

In [36]:
df_fulldata.head()

,accyear_xml,go_number,acc_number,collision_no,accyear_acc,accnb,data_source,accloc,occ_yr
0,2014.0,1008236.0,4.001008e+09,NaN,NaN,NaN,NaN,3.0,2014.0
1,2014.0,1057534.0,4.001058e+09,1533295.0,2014.0,4.001058e+09,TPS,NaN,NaN
2,2014.0,1059448.0,4.001059e+09,1533296.0,2014.0,4.001059e+09,TPS,NaN,NaN
3,2014.0,10667333.0,4.010667e+09,1549947.0,2014.0,4.010667e+09,TPS,NaN,NaN
4,2014.0,1071106.0,4.001071e+09,NaN,NaN,NaN,NaN,4.0,2014.0


In [37]:
all_xml_events = (df_fulldata.loc[~df_fulldata['accyear_xml'].isna(), :]
                  .groupby("accyear_xml")['accyear_xml'].count())
all_tps_acc_events = (df_fulldata.loc[df_fulldata['data_source'] == 'TPS', :]
                      .groupby("accyear_acc")['accnb'].count())
xml_events_not_in_acc = (
    df_fulldata.loc[~df_fulldata['accyear_xml'].isna()
                    & df_fulldata['accnb'].isna(), :].groupby("accyear_xml")['accyear_xml'].count())
# df_fulldata['accnb'].isna() is unnecessary, since there are no class 10 collisions in ACC between 2014-2019.
xml_class10_events_not_in_acc = (
    df_fulldata.loc[~df_fulldata['accyear_xml'].isna()
                    & df_fulldata['accnb'].isna()
                    & (df_fulldata['accloc'] == 10.), :].groupby("accyear_xml")['accyear_xml'].count())
xml_nonclass10_events_not_in_acc = (
    df_fulldata.loc[~df_fulldata['accyear_xml'].isna()
                    & df_fulldata['accnb'].isna()
                    & (df_fulldata['accloc'] != 10.), :].groupby("accyear_xml")['accyear_xml'].count())
xml_public_events_not_in_acc = (
    df_fulldata.loc[~df_fulldata['accyear_xml'].isna()
                    & df_fulldata['accnb'].isna()
                    & ~df_fulldata['accloc'].isin([4., 10., 98., 99.]), :].groupby("accyear_xml")['accyear_xml'].count())
acc_tps_events_not_in_xml = (
    df_fulldata.loc[df_fulldata['accyear_xml'].isna()
                    & ~df_fulldata['accnb'].isna()
                    & (df_fulldata['data_source'] == 'TPS'), :].groupby("accyear_acc")['accnb'].count())
acc_dunno_events_not_in_xml = (
    df_fulldata.loc[df_fulldata['accyear_xml'].isna()
                    & ~df_fulldata['accnb'].isna()
                    & (df_fulldata['data_source'] == 'DUNNO'), :].groupby("accyear_acc")['accnb'].count())

In [38]:
df_summary = pd.concat([all_xml_events,
                        all_tps_acc_events,
                        xml_events_not_in_acc,
                        xml_class10_events_not_in_acc,
                        xml_nonclass10_events_not_in_acc,
                        xml_public_events_not_in_acc,
                        acc_tps_events_not_in_xml,
                        acc_dunno_events_not_in_xml], axis=1).fillna(0.)

df_summary.columns = [
    'all_xml',
    'all_tps_acc',
    'xml_not_in_acc',
    'xml_class10_not_in_acc',
    'xml_nonclass10_not_in_acc',
    'xml_public_not_in_acc',
    'acc_tps_not_in_xml',
    'acc_dunno_not_in_xml'
]

df_summary

,all_xml,all_tps_acc,xml_not_in_acc,xml_class10_not_in_acc,xml_nonclass10_not_in_acc,xml_public_not_in_acc,acc_tps_not_in_xml,acc_dunno_not_in_xml
2014.0,18539,16729,1836,1341,495,185,12,0.0
2015.0,16559,14806,1761,1302,459,112,11,0.0
2016.0,13061,11507,1595,1112,483,129,49,24.0
2017.0,11342,9923,1472,1012,460,122,54,16.0
2018.0,10956,9693,1326,973,353,91,62,12.0
2019.0,10361,9154,1261,868,393,133,57,32.0


Observations (all for 2014-2019 inclusive):
- There are 80818 event reports (assuming that two files with the same filename found in different XML folders are different updates to the same collision event) in the XML folders.
- There are 71812 events in ACC with 10-digit ACCNBs (which indicates they're derived from TPS MVAs). This leaves 9251 events in the XML folders but not in ACC.
- Of these 9251, 6608 have `ACCLOC = 10`, while the others do not. According to [Evan's documentation](https://www.notion.so/bditto/CRASH-Data-Intake-Scripts-Spelunking-e2bd8895ce3d40eb9e58f24e0c0c2d76#bd6b7a2f1c664248902c41a8cfd5b3c4) only those with `ACCLOC = 10` are moved to the `ACC_ARCHIVE` table. However, the `collision_factors.accloc` table suggests that class 4, 98 and 99 could potentially also be on private property. If we remove these, there are still 772 collisions that should be in `ACC` rather than `ACC_ARCHIVE`. Jesse mentions this can come from several sources:
  - Some collisions could be outside the bounds of Toronto.
  - GO numbers are created from 911 calls, and if TPS doesn't fully consolidate all these calls there could be duplicate records for the same collision.
- There are 245 collisions that appear to be from the TPS but aren't in the XML folders.
- There are 84 collisions that don't follow the ACCNB naming conventions and don't correspond to any file in the XML folders. These only start appearing in 2016.
  - Either of the last two could be due to additional records manually created by the mail clerks to support KSI collisions that have yet to be transferred from TPS's database.